    Author: Eswar Lakshminarayanan

In [ ]:
import pandas as pd
import pickle
import datetime
import glob

In [1]:

def fn(list2d):
    flist=[]
    for each_list in list2d:
        temp =eval(each_list)
        flist.extend(temp)
    return flist

In [169]:
%%time
df = pd.read_csv("kiana_data.csv",header=0)

CPU times: user 4min 33s, sys: 34.1 s, total: 5min 7s
Wall time: 5min 52s


In [14]:
df.columns

Index(['Level', 'ClientMacAddr', 'lat', 'lng', 'localtime'], dtype='object')

In [4]:
df.Building.unique()

array(['Gallery'], dtype=object)

In [170]:
df.dtypes

Level             object
ClientMacAddr      int64
lat              float64
lng              float64
just_time         object
just_date         object
dtype: object

### `Building` column has only one unique value, drop it

In [ ]:
df.drop(['Building'], axis=1,inplace=True)

In [10]:
%%time
df.to_csv("kiana_data.csv",sep=",",index=False,header=True)

CPU times: user 18min 34s, sys: 30.1 s, total: 19min 4s
Wall time: 23min 17s


In [4]:
%%time
df = pd.read_csv("kiana_data.csv",header=0)

CPU times: user 6min 29s, sys: 38.1 s, total: 7min 7s
Wall time: 7min 12s


In [172]:
df.ClientMacAddr.nunique()

9586251

In [173]:
9586251/df.shape[0] *100

3.6860449400398934

In [174]:
df.shape

(260068750, 6)

In [175]:
df.head(5)

,Level,ClientMacAddr,lat,lng,just_time,just_date
0,L5,0,1.290866,103.851724,18:20:29.538,2018-07-31
1,L5,1,1.290866,103.851724,13:42:08.382,2018-07-31
2,L5,1,1.290866,103.851724,14:58:17.652,2018-07-31
3,L5,3,1.290866,103.851724,17:44:47.816,2018-07-31
4,L5,4,1.290866,103.851724,13:26:17.971,2018-07-31


### reduce object size `Level5 -> L5`, `Level 1 -> L1`, `Level B1 -> LB1`

In [5]:
df["Level"].replace(to_replace=["Level5"], value="L5", inplace=True)
df["Level"].replace(to_replace=["Level 1"], value="L1", inplace=True)
df["Level"].replace(to_replace=["Level B1"], value="LB1", inplace=True)

In [6]:
df.head(5)

,Level,ClientMacAddr,lat,lng,localtime
0,L5,38:ff:36:4c:1a:18,1.290866,103.851724,2018-07-31 18:20:29.538 UTC
1,L5,38:ff:36:4c:ad:23,1.290866,103.851724,2018-07-31 13:42:08.382 UTC
2,L5,38:ff:36:4c:ad:23,1.290866,103.851724,2018-07-31 14:58:17.652 UTC
3,L5,38:ff:36:4c:ae:53,1.290866,103.851724,2018-07-31 17:44:47.816 UTC
4,L5,38:ff:36:4c:b5:83,1.290866,103.851724,2018-07-31 13:26:17.971 UTC


In [7]:
df.Level.unique()

array(['L5', 'L1', 'LB1'], dtype=object)

### replace mac_id's with integer

In [6]:
ClientMacAddr = df["ClientMacAddr"].str.strip()

pandas.core.series.Series

In [7]:

ClientMacAddr.drop_duplicates(keep='first', inplace=True)

CPU times: user 1min 48s, sys: 528 ms, total: 1min 49s
Wall time: 1min 48s


#### Unique number of MAC_ID's

In [8]:
len(ClientMacAddr)

9586251

#### checking if there is any NA value

In [9]:
ClientMacAddr.isna().sum()

0

#### Using row number as integer replacement to MAC-ID's

In [13]:
ClientMacAddr = ClientMacAddr.to_frame()
ClientMacAddr.reset_index(inplace=True)
ClientMacAddr.columns
ClientMacAddr.columns = ["key","value"]
ClientMacAddr.head(5)

,key,value
0,0,38:ff:36:4c:1a:18
1,1,38:ff:36:4c:ad:23
2,3,38:ff:36:4c:ae:53
3,4,38:ff:36:4c:b5:83
4,6,38:ff:36:cc:1a:18


#### Saving MAC_ID -> Integer mapping as pickle files

In [15]:
lookup = dict(zip(ClientMacAddr.key, ClientMacAddr.value))

In [27]:
with open('lookup.pkl', 'wb') as f:
    pickle.dump(lookup, f)

In [16]:
rlookup = dict(zip(ClientMacAddr.value, ClientMacAddr.key))

In [29]:
with open('rlookup.pkl', 'wb') as f:
    pickle.dump(rlookup, f)

In [17]:
df.head(5)

,Level,ClientMacAddr,lat,lng,localtime
0,L5,38:ff:36:4c:1a:18,1.290866,103.851724,2018-07-31 18:20:29.538 UTC
1,L5,38:ff:36:4c:ad:23,1.290866,103.851724,2018-07-31 13:42:08.382 UTC
2,L5,38:ff:36:4c:ad:23,1.290866,103.851724,2018-07-31 14:58:17.652 UTC
3,L5,38:ff:36:4c:ae:53,1.290866,103.851724,2018-07-31 17:44:47.816 UTC
4,L5,38:ff:36:4c:b5:83,1.290866,103.851724,2018-07-31 13:26:17.971 UTC


In [18]:
df["ClientMacAddr"] = df["ClientMacAddr"].str.strip()

#### Replacing MAC_ID's with integers in the original dataframe

In [23]:
df["ClientMacAddr"] = df["ClientMacAddr"].map(rlookup)

In [24]:
df.head(5)

,Level,ClientMacAddr,lat,lng,localtime
0,L5,0,1.290866,103.851724,2018-07-31 18:20:29.538 UTC
1,L5,1,1.290866,103.851724,2018-07-31 13:42:08.382 UTC
2,L5,1,1.290866,103.851724,2018-07-31 14:58:17.652 UTC
3,L5,3,1.290866,103.851724,2018-07-31 17:44:47.816 UTC
4,L5,4,1.290866,103.851724,2018-07-31 13:26:17.971 UTC


In [25]:
%%time
df.to_csv("kiana_data.csv",sep=",",index=False,header=True)

CPU times: user 20min 10s, sys: 10.9 s, total: 20min 21s
Wall time: 20min 21s


### Reduction in pandas read time 23min 17s to 5min 36s

In [2]:
%%time
df = pd.read_csv("kiana_data.csv",header=0)

CPU times: user 4min 47s, sys: 37.6 s, total: 5min 25s
Wall time: 5min 36s


In [5]:
df.head(5)

,Level,ClientMacAddr,lat,lng,localtime
0,L5,0,1.290866,103.851724,2018-07-31 18:20:29.538 UTC
1,L5,1,1.290866,103.851724,2018-07-31 13:42:08.382 UTC
2,L5,1,1.290866,103.851724,2018-07-31 14:58:17.652 UTC
3,L5,3,1.290866,103.851724,2018-07-31 17:44:47.816 UTC
4,L5,4,1.290866,103.851724,2018-07-31 13:26:17.971 UTC


### Flatten data - > change data from long to wide format to create sequence of lat,lon points for each (mac_id,date)

In [ ]:
%%time
import pandas as pd
chunk_df = pd.read_csv("kiana_data.csv",header=0, chunksize=1000)

### adding to new columns to the dataframe
    
      just_date and just_time

In [184]:
%%time
just_date= []
just_time= []
counter = 0
for d in chunk_df:
    temp_df = pd.DataFrame(columns = ['just_date','just_time',"ignore"])
    counter+=1
    print("Chunk #",counter)
    temp_df[['just_date','just_time',"ignore"]]= df['localtime'].str.split(expand=True)
    just_date.extend(temp_df.just_date.values)
    just_time.extend(temp_df.just_time.values)
    print(len(just_time),len(just_date))
    print("split(" "):") 
temp_df = pd.DataFrame(columns = ['just_date','just_time',"ignore"])

'/home/opc'

In [ ]:
df["just_time"] = just_time
df["just_date"] = just_date

In [12]:
new_dir = "/home/opc/extra_space/pre_processing/"

#### forming trajectories by converting from long to wide format

In [ ]:
%%time
list_ = []
for d in chunk_df:
    temp_d = d.groupby(["ClientMacAddr","just_date"],as_index=False)["Level","lat","lng","just_time"].agg(lambda x: list(x))
    file_name = "new" + str(counter)+".csv"
    temp_d.to_csv(new_dir+file_name, index=False, header=True)

### Merge data in multiple files into single file

    - As we know Groupby operation is computation expensive given that we have 9586251 unique mac_id's

    - To tackle this, we split the reduced size data file into multiple chunks, and performed groupby by taking five files at a time
    
    - This process has to be repeated multiple times until we reduce the number of files to 1

In [60]:
%%time
import datetime
print(datetime.datetime.now())

new_dir = "/home/opc/extra_space/pre_processing9/"
file_list = list(range(0,17))
new_row_counter = 0
old_row_counter = 0
counter = 0
while len(file_list)>=5:
    counter+=1
   
    df1 = pd.read_csv(all_files[file_list.pop()])
    df2 = pd.read_csv(all_files[file_list.pop()])
    df3 = pd.read_csv(all_files[file_list.pop()])
    df4 = pd.read_csv(all_files[file_list.pop()])
    df5 = pd.read_csv(all_files[file_list.pop()])
   
    d = pd.concat([df1,df2,df3,df4,df5],axis=0,ignore_index=False)
    
    df1, df2, df3, df4, df5 = None, None, None, None, None
    
    old_row_counter += d.shape[0]
    temp_d = d.groupby(["ClientMacAddr","just_date"],as_index=False)["Level","lat","lng","just_time"].agg(lambda x: fn(list(x)))
    new_row_counter+=temp_d.shape[0]
    file_name = "new" + str(counter)+".csv"
    temp_d.to_csv(new_dir+file_name, index=False, header=True)
    
    print("# remaining",len(file_list))
    print("diif",old_row_counter - new_row_counter)

2019-04-01 08:45:13.446262
# remaining 12
diif 18745098
# remaining 7
diif 37436911
# remaining 2
diif 56172306
CPU times: user 3h 18min 44s, sys: 1min 45s, total: 3h 20min 30s
Wall time: 3h 22min 7s
